# Python pour biochimistes: soumettre des tâches dans une grappe de calcul via un script Python et un fichier de configuration

## Première partie: Introduction

Une tâche récurrente en bioinformatique est la soumission de tâches répétitives, demandant l'emploi de plusieurs fichiers contenant des informations similaires qui doivent être traité de manière similaire. Prenons, par exemple, l'alignement de plusieurs fichiers FASTQ obtenus par séquençage haute performance en utilisant un outil comme [HISAT2](https://daehwankimlab.github.io/hisat2/). Ce logiciel s'utilise minimalement ainsi pour deux fichiers *paired-end*:

```bash
hisat2 -p 4 \
-x /ou/se/trouve/index/nom_index \
-1 /ou/se/trouve/fichiers/lecture_1_R1.fastq.gz \
-2 /ou/se/trouve/fichiers/lecture_1_R2.fastq.gz \
-S /ou/se/trouve/alignement/lecture_1.sam
```

Comment interpréter ça? Comme ceci:

  - `-p 4`: Utilisons 4 coeurs de calcul (évidemment, on assume dans l'exemple qu'ils existent! Par exemple, dans un Raspberry Pi 4 ou 5)
  - `-x /ou/se/trouve/index/nom_index`: Nom de l'index utilisé, sans besoin d'ajouter les extensions .ht1 à .ht8, qui se trouve dans le répertoire `/ou/se/trouve/index`
  - `-1 /ou/se/trouve/fichiers/lecture_1_R1.fastq.gz` et
  - `-2 /ou/se/trouve/fichiers/lecture_1_R2.fastq.gz`: Noms des deux fichiers de séquences *paired-end* pour le même échantillon
  - `-S /ou/se/trouve/alignement/lecture_1.sam`: Nom du fichier en format SAM qui contiendra les alignements trouvés à partir de l'index utilisé

Un autre point important... Comme l'analyse des données en -omique demande la répétition de ces nombreuses taches de manière séquentielle, il est profitable (et souvent nécessaire) d'utiliser un type de logiciel sur le serveur qui s'appelle un système de gestion des tâches et ressources (*distributed resource management* en anglais) sur le noeud de tête d'une grappe de calcul(un ex.: Beluga ou Narval à Calcul-Québec). Pourquoi?

- C'est le genre de tâches qui dure de longues minutes ou même des heures...;
- Vous voulez vraiment rester planté devant votre écran pour tout ce temps?
- Pourquoi ne pas profiter de toutes les machines qui se cachent dans votre grappe de calcul, avec leur ressources?

Pour continuer sur le fil Calcul-Québec, le système utilisé sur les grappes s'appelle [SLURM](https://slurm.schedmd.com). Un usager soumet des scripts shell écrits en bash pour exécution et chacun de ces scripts doit contenir un en-tête spécifiant des informations importantes avant même que la commande à effectuer ne soit écrite ;-) Votre en-tête pourrait avoir ce genre de contenu (l'explication des valeurs sera décrite dans l'exemple de code ci-dessous):

```
#!/bin/bash -l 
#SBATCH --account=compteUsager 
#SBATCH --job-name hisat2_run 
#SBATCH --mail-user mon.adresse@mon.serveur 
#SBATCH --mail-type=ALL 
#SBATCH --time 4:0:0 
#SBATCH --nodes 1 
#SBATCH --cpus-per-task=4 
#SBATCH --mem 32G 
#SBATCH --output /ou/se/trouve/mes/logs
```

## Le format YAML

Ok, bon, pis après? En fait, on voit le potentiel pour l'automatisation! 

- Primo, on utilisera toujours le même index pour une opération d'alignements utilisant plusieurs fichiers;
- Secundo, on pourra organiser notre structure de fichiers pour que les fichiers d'entrée ET les fichiers de sortie soient toujours aux mêmes endroits;
- Tertio, HISAT2 et SLURM utiliseront toujours les mêmes options d'analyse pour une succession d'alignements; dans un cas comme dans l'autre, la liste d'options est longue...

Donc on voit qu'on peut isoler plusieurs éléments statiques à utiliser de manière répétitive. C'est sûr que ça pourrait s'écrire dans un script Python avec des tableaux et des dictionnaires mais on devrait refaire l'exercice à chaque fois qu'on voudra refaire le travail sur un ensemble de données différent? Une façon plus pratique: utilisons un fichier de description dans un format facilement lisible par Python, le format [YAML](https://en.wikipedia.org/wiki/YAML)

La logique du format YAML est très simple: écrivons nos informations sous forme de listes qui peuvent être elle-mêmes des listes; si nécessaire, assignons des valeurs particulières aux éléments de notre liste. Imaginons la situation suivante:

- Nos fichiers FASTQ se trouvent tous dans le répertoire `/home/moi/experience_1/donnees_brutes`
- Dans ce répertoire, vous avez vos fichiers regroupés en conditions: `control_neg`,`control_pos`,`drogue_A` et `drogue_B`
- On veut les résultats écrits dans le répertoire `/home/moi/experience_1/hisat_alignements`
- On veut écrire un fichier résumant les résultats d'alignement avec le postfixe `.summary.txt`
- On veux utiliser 4 coeurs de calcul et on spécifie des informations supplémentaires, à traiter comme des valeurs booléennes:
  - `--phred-64`: pour dire que nos fichiers présentent leur valeur *Q* en format Phred+64 (Illumina standard maintenant);
  - `--downstream-transcriptome-assembly`: pour dire de formatter les infos pour utilisation par StringTie (pour l'exemple)

Ça donne un fichier comme ça (un vrai fichier YAML est dans la section z.misc_files/demo_drm):


In [ ]:
#
# Exemple de fichier YAML pour soumission de tache
#
---
#
# YAML accepte les commentaires avec le symbol #
# devant chaque ligne!
#
# Parametre permettant de tester la  creation des scripts avant leur
# soumission. Si dry-run == false, soumettre sinon, simplement
# écrire le script pour soumission manuelle et voir si tout va bien :-)
#
dry-run: true
# Ou se trouve les séquences?
in: "/home/moi/experience_1/donnees_brutes"
# Ou écrire les résultats?
out: "/home/moi/experience_1/hisat_alignements"
#
# Quelles conditions? Les répertoires doivent être 
# ceux qui contiennent les fichiers FASTQ, sous
# /home/moi/experience_1/donnees_brutes
#
# Remarquez l'utilisation des espaces pour l'indentation...
#
conditions:
  - control_neg
  - control_pos
  - drogue_A
  - drogue_B
#
# Ici, on specifie que ces options seront utilisées
# pour HISAT2
#
# Ma preference est d'utiliser les noms longs pour les parametres
# car ensuite on peut les utiliser tel quel dans le script a venir.
#
hisat2_static:
  - x: /ou/se/trouve/hisat2/index
  - phred-64: true
  - rna-strandness: RF
  - downstream-transcriptome-assembly: true
  #
  # Ici, on déclare une variable appelée cpu avec la caractère &
  # Pourquoi? Pour récupérer la valeur (4) dans la section des commandes
  # du système de gestion de taches.
  #
  - threads: &cpu
      val: 4
#
# Ça va nous servir pour écrire l'en-tete du script bash qui serait soumis
# au système SLURM 
#
slurm_static:
  # compte usager dans SLURM
  - account: compteUsager
  # nom donné à la tache; on pourra jouer avec la valeur dasn le script
  # d'écriture python
  - job-name: hisat2_run 
  # à qui envoyer les courriels
  - mail-user: mon.adresse@mon.serveur 
  # on envoit tous les messages, erreur comme complétion
  - mail-type: ALL 
  # temps maximum alloué à la tache (h:m:s)
  - time: 4:0:0 
  # les cpus habitent sur un seul noeud de calcul
  - nodes: 1 
  # on prend &cpu, tel que défini en amont
  - cpus-per-task: *cpu 
  # chaque tache ne doit pas prendre plus de 8G de memoire
  - mem: 8G 
  # on écrit toute sortie de l'exécution dans ce répertoire
  # ce n'est pas tout à fait fini car on prendra la valeur 
  # pour construire le chemin final
  - output: /ou/se/trouve/mes/logs

```